In [2]:
from scapy.all import *
# Read packets from your pcap
packets = rdpcap("7.pcap")
print(len(packets))  # how many packets in the file


801440


In [4]:
dns_packets = [pkt for pkt in packets if pkt.haslayer(DNS)]
print(f"Total DNS packets: {len(dns_packets)}")


Total DNS packets: 32


In [5]:
pkt = dns_packets[0]
pkt.show()   # human-readable breakdown


###[ Ethernet ]###
  dst       = 01:00:5e:00:00:fb
  src       = 14:10:9f:cf:e1:71
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 569
     id        = 50734
     flags     = 
     frag      = 0
     ttl       = 255
     proto     = udp
     chksum    = 0xe166
     src       = 172.16.133.18
     dst       = 224.0.0.251
     \options   \
###[ UDP ]###
        sport     = mdns
        dport     = mdns
        len       = 549
        chksum    = 0xd460
###[ DNS ]###
           id        = 0
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 0
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 14
           ancount   = 6
           nscount   = 0
           arcount   = 0
           \qd        \
            |###[ DNS Question Record ]###
            |  qname    

In [6]:
dns_layer = pkt[DNS]

print("Transaction ID:", dns_layer.id)
print("Is response?:", dns_layer.qr)    # 0=query, 1=response
print("Opcode:", dns_layer.opcode)
print("Number of Questions:", dns_layer.qdcount)
print("Number of Answers:", dns_layer.ancount)

# Question section
if dns_layer.qd:
    print("Query Name:", dns_layer.qd.qname.decode())
    print("Query Type:", dns_layer.qd.qtype)
    print("Query Class:", dns_layer.qd.qclass)

# Answer section
if dns_layer.an:
    for i in range(dns_layer.ancount):
        ans = dns_layer.an[i]
        print(f"Answer {i+1}: {ans.rrname.decode()} → {ans.rdata}")


Transaction ID: 0
Is response?: 0
Opcode: 0
Number of Questions: 14
Number of Answers: 6
Query Name: _apple-mobdev._tcp.local.
Query Type: 12
Query Class: 1
Answer 1: _ipp._tcp.local. → b'HP LaserJet P3010 Series [8E108F]._ipp._tcp.local.'
Answer 2: _ipp._tcp.local. → b'HP LaserJet P3010 Series [0F1041]._ipp._tcp.local.'
Answer 3: _ipp._tcp.local. → b'Samsung CLP-775 Series (SEC001599AFF4AD)._ipp._tcp.local.'
Answer 4: _printer._tcp.local. → b'HP LaserJet P3010 Series [8E108F]._printer._tcp.local.'
Answer 5: _printer._tcp.local. → b'HP LaserJet P3010 Series [0F1041]._printer._tcp.local.'
Answer 6: _printer._tcp.local. → b'Samsung CLP-775 Series (SEC001599AFF4AD)._printer._tcp.local.'


In [7]:
for i, pkt in enumerate(dns_packets[:5]):  # just first 5
    print(f"\nPacket #{i+1}")
    pkt[DNS].show()



Packet #1
###[ DNS ]###
  id        = 0
  qr        = 0
  opcode    = QUERY
  aa        = 0
  tc        = 0
  rd        = 0
  ra        = 0
  z         = 0
  ad        = 0
  cd        = 0
  rcode     = ok
  qdcount   = 14
  ancount   = 6
  nscount   = 0
  arcount   = 0
  \qd        \
   |###[ DNS Question Record ]###
   |  qname     = b'_apple-mobdev._tcp.local.'
   |  qtype     = PTR
   |  unicastresponse= 0
   |  qclass    = IN
   |###[ DNS Question Record ]###
   |  qname     = b'_airplay._tcp.local.'
   |  qtype     = PTR
   |  unicastresponse= 0
   |  qclass    = IN
   |###[ DNS Question Record ]###
   |  qname     = b'_raop._tcp.local.'
   |  qtype     = PTR
   |  unicastresponse= 0
   |  qclass    = IN
   |###[ DNS Question Record ]###
   |  qname     = b'_odisk._tcp.local.'
   |  qtype     = PTR
   |  unicastresponse= 0
   |  qclass    = IN
   |###[ DNS Question Record ]###
   |  qname     = b'_afpovertcp._tcp.local.'
   |  qtype     = PTR
   |  unicastresponse= 0
   |  qclass

In [8]:
for pkt in dns_packets[:5]:
    if pkt.haslayer(DNSQR):
        print("Query:", pkt[DNSQR].qname.decode())
    if pkt.haslayer(DNSRR):
        print("Answer:", pkt[DNSRR].rdata)


Query: _apple-mobdev._tcp.local.
Answer: b'HP LaserJet P3010 Series [8E108F]._ipp._tcp.local.'
Query: _apple-mobdev._tcp.local.
Answer: b'HP LaserJet P3010 Series [8E108F]._ipp._tcp.local.'
Answer: b'John Vigeant\xe2\x80\x99s MacBook Pro._smb._tcp.local.'
Answer: b'JLTM7._rfb._tcp.local.'
Answer: b'Sean Armstrong\xe2\x80\x99s MacBook Pro._afpovertcp._tcp.local.'


## dpkt

In [13]:
import dpkt

with open("7.pcap", "rb") as f:
    pcap = dpkt.pcap.Reader(f)
    for ts, buf in pcap:
        eth = dpkt.ethernet.Ethernet(buf)     # Parse Ethernet frame

        if not isinstance(eth.data, dpkt.ip.IP):
            continue

        ip = eth.data
        if not isinstance(ip.data, dpkt.udp.UDP):
            continue

        udp = ip.data

        # DNS is usually UDP port 53
        if udp.sport != 53 and udp.dport != 53:
            continue

        try:
            dns = dpkt.dns.DNS(udp.data)
        except (dpkt.dpkt.NeedData, dpkt.dpkt.UnpackError):
            continue



In [14]:
print(f"\nTimestamp: {ts}")
print("Transaction ID:", dns.id)
print("Is Response?:", dns.qr)    # 0=query, 1=response
print("Opcode:", dns.opcode)
print("Questions:", len(dns.qd))
print("Answers:", len(dns.an))

# Questions
for q in dns.qd:
    print("  Query:", q.name, "Type:", q.type)

# Answers
for ans in dns.an:
    print("  Answer:", ans.name, "→", ans.rdata)



Timestamp: 1295981655.556513
Transaction ID: 0
Is Response?: 0
Opcode: 0
Questions: 1
Answers: 0
  Query: linkedin.com Type: 1


In [17]:
# import dpkt

# def print_dns_packet(ts, buf):
#     eth = dpkt.ethernet.Ethernet(buf)
#     print("Ethernet:")
#     print(f"  Src MAC: {eth.src.hex(':')}")
#     print(f"  Dst MAC: {eth.dst.hex(':')}")
#     if not isinstance(eth.data, dpkt.ip.IP):
#         return

#     ip = eth.data
#     print("  IP:")
#     print(f"    Src: {ip.src.hex('.')}")
#     print(f"    Dst: {ip.dst.hex('.')}")
#     print(f"    Protocol: {ip.p}")

#     if not isinstance(ip.data, dpkt.udp.UDP):
#         return

#     udp = ip.data
#     print("    UDP:")
#     print(f"      Src Port: {udp.sport}")
#     print(f"      Dst Port: {udp.dport}")

#     # Check if it's DNS
#     if udp.sport != 53 and udp.dport != 53:
#         return

#     try:
#         dns = dpkt.dns.DNS(udp.data)
#     except (dpkt.dpkt.NeedData, dpkt.dpkt.UnpackError):
#         return

#     print("      DNS:")
#     print(f"        Transaction ID: {dns.id}")
#     print(f"        Flags: qr={dns.qr}, opcode={dns.opcode}, rcode={dns.rcode}")
#     print(f"        Questions: {len(dns.qd)}")
#     print(f"        Answers: {len(dns.an)}")

#     # Questions
#     for q in dns.qd:
#         print(f"          Q: {q.name} (Type={q.type}, Class={q.cls})")

#     # Answers
#     for ans in dns.an:
#         print(f"          A: {ans.name} → {ans.rdata}")

# # -------- Main loop --------
# with open("7.pcap", "rb") as f:
#     pcap = dpkt.pcap.Reader(f)
#     for i, (ts, buf) in enumerate(pcap):
#         print(f"\nPacket #{i+1} @ {ts}")
#         print_dns_packet(ts, buf)
